In [1]:
%pip install html5lib
%pip install webdriver-manager
%pip install selenium


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from time import sleep
import pandas as pd
from io import StringIO
from functools import partial
import matplotlib.pyplot as plt
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common import NoSuchElementException,StaleElementReferenceException
from matplotlib import style
from webdriver_manager.chrome import ChromeDriverManager




print(plt.style.available)
%matplotlib inline
driver = webdriver.Chrome()

url='https://zu.fang.com/house-a015277-b07195/'
driver.get(url)

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'petroff10', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd

# 初始化浏览器驱动
driver = webdriver.Chrome()  # 需确保已配置ChromeDriver
url ='https://zu.fang.com/house-a015277-b07195/'
driver.get(url)

# 初始化存储所有页数据的列表
all_data = []
Num_Pages = 20  # 最大爬取页数
current_page = 0

while current_page < Num_Pages:
    try:
        # 定位表格容器
        table = driver.find_element('class name', 'houseList')
        # 提取行（dl标签）
        rows = table.find_elements(By.TAG_NAME, 'dl')
        
        # 提取当前页的每行数据
        page_data = []
        for row in rows:
            # 提取p.tel_shop
            try:
                tel_shop = row.find_element('class name', 'font15').text
            except NoSuchElementException:
                tel_shop = None
            
            # 提取dd.price_right
            try:
                price_right = row.find_element('class name', 'moreInfo').text
            except NoSuchElementException:
                price_right = None
            
            # 添加非空行
            if tel_shop or price_right:
                page_data.append([tel_shop, price_right])
        
        # 将当前页数据添加到总列表
        all_data.extend(page_data)
        print(f"已爬取第{current_page + 1}页，累计{len(all_data)}条数据")
        
        # 点击下一页
        # 更推荐的写法
        next_page = driver.find_element(By.XPATH, "//a[text()='下一页']")
        next_page.click()
        current_page += 1
        
    except NoSuchElementException as e:
        print(f"未找到元素：{e}，可能已到最后一页")
        break
    except Exception as e:
        print(f"爬取出错：{e}")
        break

# 关闭浏览器

# 生成最终DataFrame
df = pd.DataFrame(all_data, columns=['tel_shop', 'price_right'])
print("爬取完成，数据预览：")
print(df.head())

# 可选：保存为CSV
# df.to_csv('爬取结果.csv', index=False, encoding='utf-8-sig')
    

已爬取第1页，累计60条数据
已爬取第2页，累计120条数据
已爬取第3页，累计180条数据
已爬取第4页，累计240条数据
已爬取第5页，累计300条数据
已爬取第6页，累计360条数据
已爬取第7页，累计420条数据
已爬取第8页，累计424条数据
未找到元素：Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[text()='下一页']"}
  (Session info: chrome=141.0.7390.77); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
	GetHandleVerifier [0x0x7ff63f70e955+80021]
	GetHandleVerifier [0x0x7ff63f70e9b0+80112]
	(No symbol) [0x0x7ff63f49060f]
	(No symbol) [0x0x7ff63f4e8854]
	(No symbol) [0x0x7ff63f4e8b1c]
	(No symbol) [0x0x7ff63f53c927]
	(No symbol) [0x0x7ff63f51126f]
	(No symbol) [0x0x7ff63f53968a]
	(No symbol) [0x0x7ff63f511003]
	(No symbol) [0x0x7ff63f4d95d1]
	(No symbol) [0x0x7ff63f4da3f3]
	GetHandleVerifier [0x0x7ff63f9cdd0d+2960461]
	GetHandleVerifier [0x0x7ff63f9c7fca+2936586]
	GetHandleVerifier [0x0x7ff63f9e8a07+3070279]
	GetHandleVerifier [0x0x7ff63f72843e+185214]
	GetHandleVe

In [4]:
import pandas as pd
import re


pattern_m2 = r'(\d+(\.\d+)?)㎡'       # 匹配 xx㎡ 或 xx.x㎡
pattern_yuan = r'(\d+(\.\d+)?)元/月'  # 匹配 xx元/月

# 提取数据并转换为数值类型（方便后续计算）
df['m2'] = df['tel_shop'].str.extract(pattern_m2)[0].astype(float)
df['rent_month'] = df['price_right'].str.extract(pattern_yuan)[0].astype(float)

# 只保留目标三列
result_df = df[['m2',  'rent_month']]
print(result_df)

        m2  rent_month
0     18.0      2300.0
1     16.0      2100.0
2    122.0     12000.0
3    122.0     12000.0
4    233.0     28000.0
..     ...         ...
419   99.0      8600.0
420   88.0      5700.0
421   58.0      5990.0
422  233.0     30000.0
423  152.0     14500.0

[424 rows x 2 columns]


In [6]:
!pip install openpyxl


In [7]:
result_df.to_excel('xierqi_zu.xlsx', index=False, engine='openpyxl')